# Hyperparameter Tuning

This notebook is a place for us to tune our the hyperparameters for both the lightgbm and catboost models for both the standard and complex name sets.

We will tune each model on each data set and then do a final comparisson. So this will in part inform us of the best parameter choices and the best model choice.

In [1]:
!pip install lightgbm
!pip install sqlalchemy
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 29.0 MB/s eta 0:00:00


In [2]:
import json
import gc
import re
import random
import matplotlib.pyplot as plt
import lightgbm as lgb
import pandas as pd
import numpy as np
import sqlite3
import optuna
from catboost import CatBoostRanker, Pool
from datetime import datetime
from collections import defaultdict
from sqlalchemy import select, func
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import ndcg_score

In [5]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


## Standard Nameset

We will begin on the standard set.

In [6]:
# Paths to files in Drive
db_path = "/content/drive/MyDrive/Colab Notebooks/database.db"
train_ids_path = "/content/train_std_ids.csv"
val_ids_path = "/content/validation_std_ids.csv"

Pull whole standard dataset from SQL db into pandas frame.

In [5]:
conn = sqlite3.connect(db_path)  # or your local path
full_df = pd.read_sql_query("SELECT * FROM feature_matrix", conn)

Saved the training and validation IDs in a CSV so we can split the data set.

In [6]:
train_ids = pd.read_csv(train_ids_path)["train_ids"].dropna().astype(int).tolist()
val_ids = pd.read_csv(val_ids_path)["validation_ids"].dropna().astype(int).tolist()

Validation dataset pulled from full set, remianing is training set. This is to ensure that we don't train on validation set.

In [7]:
val_ids_set = set(val_ids)
val_df = full_df[full_df["clean_row_id"].isin(val_ids_set)]
full_df = full_df[~full_df["clean_row_id"].isin(val_ids_set)]

This was pulled from the initial model development notebook.

In [13]:
def compute_ranking_metrics(df, k=3):
    # Group
    grouped = df.groupby("clean_row_id")

    # Get top 1 and calculate accuracy
    top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
    acc1 = (top1["label"] == 1).mean()

    # Same with recall
    topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
    recall_k = topk.groupby("clean_row_id")["label"].max().mean()

    # MMR
    def reciprocal_rank(g):
        sorted_g = g.sort_values("score", ascending=False).reset_index()
        match = sorted_g[sorted_g["label"] == 1]
        return 1.0 / (match.index[0] + 1) if not match.empty else 0.0

    mrr = grouped.apply(reciprocal_rank).mean()
    return acc1, recall_k, mrr

We will define our train and val set here so we can reuse on LightGBM tuning.

In [9]:
# Training set
X_train = full_df.drop(
    columns=["label", "clean_row_id", "investor", "firm", "template_id"]
)
y_train = full_df["label"]
train_group_sizes = full_df.groupby("clean_row_id").size().tolist()

# Validation set
X_val = val_df.drop(
    columns=["label", "clean_row_id", "investor", "firm", "template_id"]
)
y_val = val_df["label"]
val_group_sizes = val_df.groupby("clean_row_id").size().tolist()

We define a catboost objective function to tune our parameters. We want to do a full sweep on hyperparameters to make sure our model performs the best it can.

In [10]:
def catboost_objective(trial):
    params = {
        "loss_function": "YetiRank",
        "eval_metric": "NDCG:top=3",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "random_strength": trial.suggest_float("random_strength", 0, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "grow_policy": trial.suggest_categorical(
            "grow_policy", ["SymmetricTree", "Lossguide"]
        ),
        "iterations": 100,
        "early_stopping_rounds": 5,
        "random_seed": 42,
        "verbose": False,
        "task_type": "CPU",
    }

    # CatBoost pools
    train_pool = Pool(
        data=X_train,
        label=y_train,
        group_id=np.repeat(np.arange(len(train_group_sizes)), train_group_sizes),
    )
    val_pool = Pool(
        data=X_val,
        label=y_val,
        group_id=np.repeat(np.arange(len(val_group_sizes)), val_group_sizes),
    )

    # Train
    model = CatBoostRanker(**params)
    model.fit(train_pool, eval_set=val_pool)

    # Score
    val_df_copy = val_df.copy()
    val_df_copy["score"] = model.predict(val_pool)
    acc1, recall3, mrr = compute_ranking_metrics(val_df_copy, k=3)

    return 1 - recall3  # minimize (maximize recall@3)

And the same for LightGBM.

In [11]:
def lightgbm_objective(trial):
    params = {
        "objective": "lambdarank",
        "metric": "ndcg",
        "ndcg_eval_at": [3],
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 0, 5),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 5),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "verbose": -1,
    }

    train_set = lgb.Dataset(X_train, label=y_train, group=train_group_sizes)
    val_set = lgb.Dataset(X_val, label=y_val, group=val_group_sizes)

    model = lgb.train(
        params,
        train_set,
        num_boost_round=100,
        valid_sets=[val_set],
        callbacks=[
            lgb.early_stopping(stopping_rounds=30),
            lgb.log_evaluation(period=0),
        ],
    )

    preds = model.predict(X_val, num_iteration=model.best_iteration)
    val_df_copy = val_df.copy()
    val_df_copy["score"] = preds

    acc1, recall3, mrr = compute_ranking_metrics(val_df_copy, k=3)
    return 1 - recall3

Lets start our sweep

In [12]:
# Run the sweep
print("Running CatBoost tuning...")
catboost_study = optuna.create_study(direction="minimize", study_name="catboost")
catboost_study.optimize(catboost_objective, n_trials=20)
# Save to drive
with open("/content/catboost_std_best.json", "w") as f:
    json.dump(catboost_study.best_params, f)

[I 2025-07-16 15:16:02,320] A new study created in memory with name: catboost


Running CatBoost tuning...


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

In [13]:
print("Running LightGBM tuning...")
lgbm_study = optuna.create_study(direction="minimize", study_name="lightgbm")
lgbm_study.optimize(lightgbm_objective, n_trials=20)
# Save to drive
with open("/content/light_gbm_std_best.json", "w") as f:
    json.dump(lgbm_study.best_params, f)

[I 2025-07-16 15:51:10,804] A new study created in memory with name: lightgbm


Running LightGBM tuning...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's ndcg@3: 0.960617


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[12]	valid_0's ndcg@3: 0.942363


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

...

/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[94]	valid_0's ndcg@3: 0.963846


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's ndcg@3: 0.961642


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

In [14]:
print("\nBest CatBoost:")
print(catboost_study.best_params)
print("Recall@3:", 1 - catboost_study.best_value)

print("\nBest LightGBM:")
print(lgbm_study.best_params)
print("Recall@3:", 1 - lgbm_study.best_value)


Best CatBoost:
{'learning_rate': 0.13275757957731918, 'depth': 6, 'l2_leaf_reg': 7.142519331365267, 'random_strength': 3.395785387976391, 'min_data_in_leaf': 84, 'subsample': 0.9048958560910838, 'colsample_bylevel': 0.511123337191838, 'grow_policy': 'Lossguide'}
Recall@3: 0.9961381677751555

Best LightGBM:
{'learning_rate': 0.19426829578921662, 'num_leaves': 64, 'min_data_in_leaf': 75, 'feature_fraction': 0.9047338201190456, 'bagging_fraction': 0.9624860112209651, 'lambda_l1': 0.9753561860776749, 'lambda_l2': 4.367377782782343, 'bagging_freq': 3, 'max_depth': 10}
Recall@3: 0.9972108989487235


Now we have our parameters, let's train both models and see which performs the best.

In [15]:
def train_model_lightGBM(
    parameters: dict,
    n_rounds: int = 500,
    lr_decay_gamma: float = 0.95,
    val_df_input
):
    # Load validation set
    lgb_val = lgb.Dataset(X_val, label=y_val, group=val_group_sizes, free_raw_data=False)

    # Load full training set
    lgb_train = lgb.Dataset(X_train, label=y_train, group=train_group_sizes, free_raw_data=False)

    # Learning rate schedule
    def lr_decay(current_round):
        return parameters["learning_rate"] * (lr_decay_gamma ** current_round)

    # Train model
    model = lgb.train(
        params=parameters,
        train_set=lgb_train,
        num_boost_round=n_rounds,
        valid_sets=[lgb_train, lgb_val],
        valid_names=["train", "val"],
        callbacks=[
            lgb.reset_parameter(learning_rate=lr_decay),
            lgb.early_stopping(stopping_rounds=500),
            lgb.log_evaluation(period=1)
        ]
    )

    # Predict and Evaluate
    preds = model.predict(X_val, num_iteration=model.best_iteration)
    val_df["score"] = preds

    acc1, recall3, mrr = compute_ranking_metrics(val_df, k=3)

    print("\Evaluation Metrics (Validation Set):")
    print(f"Accuracy@1 : {acc1:.4f}")
    print(f"Recall@3   : {recall3:.4f}")
    print(f"MRR        : {mrr:.4f}")

    return model


In [11]:
def train_catboost_model(parameters: dict, n_rounds: int = 500, val_df_input=None):
    # Training group_id
    train_group_id = np.repeat(np.arange(len(train_group_sizes)), train_group_sizes)

    # Validation group_id
    val_group_id = np.repeat(np.arange(len(val_group_sizes)), val_group_sizes)
    # Create Pools for CatBoost
    train_pool = Pool(data=X_train, label=y_train, group_id=train_group_id)
    val_pool = Pool(data=X_val, label=y_val, group_id=val_group_id)

    # Train the model
    model = CatBoostRanker(iterations=n_rounds, **parameters)

    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=True)

    # Predict and evaluate
    preds = model.predict(val_pool)
    val_df = val_df_input.copy()
    val_df["score"] = preds

    acc1, recall3, mrr = compute_ranking_metrics(val_df, k=3)

    print("\nEvaluation Metrics (Validation Set):")
    print(f"Accuracy@1 : {acc1:.4f}")
    print(f"Recall@3   : {recall3:.4f}")
    print(f"MRR        : {mrr:.4f}")

    return model

In [20]:
# Static/default parameters that LightGBM expects
base_params = {
    "objective": "lambdarank",
    "metric": ["ndcg"],
    "eval_at": [1, 3],
    "boosting_type": "gbdt",
    "verbosity": -1,
    "force_row_wise": True,
}

# Load from file
with open("light_gbm_std_best.json", "r") as f:
    best_params_from_json = json.load(f)

# Merge base + Optuna best
final_params = {**base_params, **best_params_from_json}

train_model_lightGBM(parameters=final_params, n_rounds=1000, lr_decay_gamma=0.95)

[1]	train's ndcg@1: 0.876358	train's ndcg@3: 0.94871	val's ndcg@1: 0.67675	val's ndcg@3: 0.853679
Training until validation scores don't improve for 500 rounds
[2]	train's ndcg@1: 0.924617	train's ndcg@3: 0.970516	val's ndcg@1: 0.852106	val's ndcg@3: 0.940443
[3]	train's ndcg@1: 0.929556	train's ndcg@3: 0.972374	val's ndcg@1: 0.877566	val's ndcg@3: 0.949844
[4]	train's ndcg@1: 0.929909	train's ndcg@3: 0.972571	val's ndcg@1: 0.879783	val's ndcg@3: 0.951019
[5]	train's ndcg@1: 0.930351	train's ndcg@3: 0.972744	val's ndcg@1: 0.879282	val's ndcg@3: 0.950731
[6]	train's ndcg@1: 0.930717	train's ndcg@3: 0.972866	val's ndcg@1: 0.881785	val's ndcg@3: 0.951585
[7]	train's ndcg@1: 0.93112	train's ndcg@3: 0.973051	val's ndcg@1: 0.88393	val's ndcg@3: 0.95257
[8]	train's ndcg@1: 0.93123	train's ndcg@3: 0.973103	val's ndcg@1: 0.884074	val's ndcg@3: 0.952595
[9]	train's ndcg@1: 0.931496	train's ndcg@3: 0.97322	val's ndcg@1: 0.885003	val's ndcg@3: 0.953002
[10]	train's ndcg@1: 0.931746	train's ndcg@3:

/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)


\Evaluation Metrics (Validation Set):
Accuracy@1 : 0.9005
Recall@3   : 0.9966
MRR        : 0.9471


/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mrr = grouped.apply(reciprocal_rank).mean()


Strong results. The correct prediction is almost always in the top 3 ranked items and is in the top 1 stop 90% of the time.

In [24]:
# Base/defaults for CatBoost
base_params = {
    "loss_function": "YetiRank",
    "eval_metric": "NDCG:top=3",
    "random_seed": 42,
    "task_type": "CPU",  # or "GPU"
}

# Load from file
with open("catboost_std_best.json", "r") as f:
    best_params_from_json = json.load(f)

# Merge base + Optuna best
final_params = {**base_params, **best_params_from_json}

best_catboost_model = train_catboost_model(
    parameters=final_params, n_rounds=1000, val_df_input=val_df
)

0:	test: 0.8931327	best: 0.8931327 (0)	total: 2.67s	remaining: 44m 29s
1:	test: 0.9371556	best: 0.9371556 (1)	total: 5.34s	remaining: 44m 22s
2:	test: 0.9366829	best: 0.9371556 (1)	total: 7.92s	remaining: 43m 52s
3:	test: 0.9369486	best: 0.9371556 (1)	total: 11s	remaining: 45m 29s
4:	test: 0.9365407	best: 0.9371556 (1)	total: 13.7s	remaining: 45m 35s
5:	test: 0.9394015	best: 0.9394015 (5)	total: 16.8s	remaining: 46m 27s
6:	test: 0.9397957	best: 0.9397957 (6)	total: 19.7s	remaining: 46m 41s
7:	test: 0.9400222	best: 0.9400222 (7)	total: 22.4s	remaining: 46m 12s
8:	test: 0.9400545	best: 0.9400545 (8)	total: 24.8s	remaining: 45m 30s
9:	test: 0.9403287	best: 0.9403287 (9)	total: 27.6s	remaining: 45m 29s
10:	test: 0.9410855	best: 0.9410855 (10)	total: 30.2s	remaining: 45m 14s
11:	test: 0.9444052	best: 0.9444052 (11)	total: 32.5s	remaining: 44m 38s
12:	test: 0.9449433	best: 0.9449433 (12)	total: 34.8s	remaining: 44m 2s
13:	test: 0.9430693	best: 0.9449433 (12)	total: 37.5s	remaining: 43m 58s
1

/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)



Evaluation Metrics (Validation Set):
Accuracy@1 : 0.9010
Recall@3   : 0.9961
MRR        : 0.9473


/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mrr = grouped.apply(reciprocal_rank).mean()


Catboost has a slight edge here. Lets see how it performs on the complex set.

# Complex Set

Now we will tune the complex model as well. We will delete the previous data to save on RAM first.

In [ ]:
# Delete training-related objects
del full_df, X_train, y_train, train_group_sizes
del val_df, X_val, y_val, val_group_sizes

# Run garbage collection
gc.collect()

In [7]:
# Get new ids
train_ids_path = "/content/train_com_ids.csv"
val_ids_path = "/content/validation_com_ids.csv"
train_ids = pd.read_csv(train_ids_path)["train_ids"].dropna().astype(int).tolist()
val_ids = pd.read_csv(val_ids_path)["validation_ids"].dropna().astype(int).tolist()

# Reconnect and load the second dataset
conn = sqlite3.connect(db_path)
full_df = pd.read_sql_query("SELECT * FROM feature_matrix_complex", conn)
val_ids_set = set(val_ids)
val_df = full_df[full_df["clean_row_id"].isin(val_ids_set)]
full_df = full_df[~full_df["clean_row_id"].isin(val_ids_set)]


# Rebuild train/val splits
X_train = full_df.drop(
    columns=["label", "clean_row_id", "investor", "firm", "template_id"]
)
y_train = full_df["label"]
train_group_sizes = full_df.groupby("clean_row_id").size().tolist()
X_val = val_df.drop(
    columns=["label", "clean_row_id", "investor", "firm", "template_id"]
)
y_val = val_df["label"]
val_group_sizes = val_df.groupby("clean_row_id").size().tolist()

Now lets re run the full sweep.

In [26]:
# Run the sweep
print("Running CatBoost tuning...")
catboost_study = optuna.create_study(direction="minimize", study_name="catboost")
catboost_study.optimize(catboost_objective, n_trials=50)
# Save to drive
with open("/content/catboost_com_best.json", "w") as f:
    json.dump(catboost_study.best_params, f)

[I 2025-07-16 17:29:25,081] A new study created in memory with name: catboost


Running CatBoost tuning...


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

In [27]:
print("Running LightGBM tuning...")
lgbm_study = optuna.create_study(direction="minimize", study_name="lightgbm")
lgbm_study.optimize(lightgbm_objective, n_trials=50)
# Save to drive
with open("/content/light_gbm_com_best.json", "w") as f:
    json.dump(lgbm_study.best_params, f)

[I 2025-07-16 22:46:54,491] A new study created in memory with name: lightgbm


Running LightGBM tuning...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's ndcg@3: 0.939641


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's ndcg@3: 0.912125


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

...

/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[84]	valid_0's ndcg@3: 0.938882


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[61]	valid_0's ndcg@3: 0.936843


/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply

In [28]:
print("\nBest CatBoost:")
print(catboost_study.best_params)
print("Recall@3:", 1 - catboost_study.best_value)

print("\nBest LightGBM:")
print(lgbm_study.best_params)
print("Recall@3:", 1 - lgbm_study.best_value)


Best CatBoost:
{'learning_rate': 0.1329416174119581, 'depth': 8, 'l2_leaf_reg': 6.628807908190987, 'random_strength': 4.881288964388853, 'min_data_in_leaf': 100, 'subsample': 0.708915364734102, 'colsample_bylevel': 0.6444718826631752, 'grow_policy': 'Lossguide'}
Recall@3: 0.985

Best LightGBM:
{'learning_rate': 0.14925362110050383, 'num_leaves': 61, 'min_data_in_leaf': 39, 'feature_fraction': 0.9509281191383269, 'bagging_fraction': 0.9584197752403462, 'lambda_l1': 2.766186105296192, 'lambda_l2': 4.511482480952715, 'bagging_freq': 1, 'max_depth': 9}
Recall@3: 0.99125


In [29]:
# Static/default parameters that LightGBM expects
base_params = {
    "objective": "lambdarank",
    "metric": ["ndcg"],
    "eval_at": [1, 3],
    "boosting_type": "gbdt",
    "verbosity": -1,
    "force_row_wise": True,
}

# Load from file
with open("light_gbm_com_best.json", "r") as f:
    best_params_from_json = json.load(f)

# Merge base + Optuna best
final_params = {**base_params, **best_params_from_json}

train_model_lightGBM(parameters=final_params, n_rounds=1000, lr_decay_gamma=0.95)

[1]	train's ndcg@1: 0.712771	train's ndcg@3: 0.881039	val's ndcg@1: 0.49875	val's ndcg@3: 0.697153
Training until validation scores don't improve for 500 rounds
[2]	train's ndcg@1: 0.916362	train's ndcg@3: 0.966712	val's ndcg@1: 0.69125	val's ndcg@3: 0.83077
[3]	train's ndcg@1: 0.934003	train's ndcg@3: 0.973692	val's ndcg@1: 0.7825	val's ndcg@3: 0.894476
[4]	train's ndcg@1: 0.936357	train's ndcg@3: 0.974752	val's ndcg@1: 0.79125	val's ndcg@3: 0.898494
[5]	train's ndcg@1: 0.938536	train's ndcg@3: 0.975715	val's ndcg@1: 0.7925	val's ndcg@3: 0.898792
[6]	train's ndcg@1: 0.942472	train's ndcg@3: 0.977236	val's ndcg@1: 0.79375	val's ndcg@3: 0.899714
[7]	train's ndcg@1: 0.943334	train's ndcg@3: 0.977538	val's ndcg@1: 0.795	val's ndcg@3: 0.899848
[8]	train's ndcg@1: 0.945068	train's ndcg@3: 0.978197	val's ndcg@1: 0.795	val's ndcg@3: 0.900176
[9]	train's ndcg@1: 0.946764	train's ndcg@3: 0.978825	val's ndcg@1: 0.8025	val's ndcg@3: 0.903107
[10]	train's ndcg@1: 0.948822	train's ndcg@3: 0.979593	

/tmp/ipython-input-8-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-8-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)


\Evaluation Metrics (Validation Set):
Accuracy@1 : 0.8137
Recall@3   : 0.9712
MRR        : 0.8911


/tmp/ipython-input-8-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mrr = grouped.apply(reciprocal_rank).mean()


In [14]:
# Base/defaults for CatBoost
base_params = {
    "loss_function": "YetiRank",
    "eval_metric": "NDCG:top=3",
    "random_seed": 42,
    "task_type": "CPU",  # or "GPU"
}

# Load from file
best_params_from_json = {
    "learning_rate": 0.1329416174119581,
    "depth": 8,
    "l2_leaf_reg": 6.628807908190987,
    "random_strength": 4.881288964388853,
    "min_data_in_leaf": 100,
    "subsample": 0.708915364734102,
    "colsample_bylevel": 0.6444718826631752,
    "grow_policy": "Lossguide",
}

# Merge base + Optuna best
final_params = {**base_params, **best_params_from_json}

best_catboost_model = train_catboost_model(
    parameters=final_params, n_rounds=1000, val_df_input=val_df
)

0:	test: 0.7810507	best: 0.7810507 (0)	total: 8.58s	remaining: 2h 22m 50s
1:	test: 0.8003522	best: 0.8003522 (1)	total: 17.3s	remaining: 2h 24m 9s
2:	test: 0.8017659	best: 0.8017659 (2)	total: 26.1s	remaining: 2h 24m 43s
3:	test: 0.8056795	best: 0.8056795 (3)	total: 34.4s	remaining: 2h 22m 39s
4:	test: 0.8048909	best: 0.8056795 (3)	total: 43.3s	remaining: 2h 23m 32s
5:	test: 0.8048909	best: 0.8056795 (3)	total: 52s	remaining: 2h 23m 42s
6:	test: 0.8073612	best: 0.8073612 (6)	total: 1m	remaining: 2h 23m 45s
7:	test: 0.8083136	best: 0.8083136 (7)	total: 1m 9s	remaining: 2h 22m 35s
8:	test: 0.8084772	best: 0.8084772 (8)	total: 1m 17s	remaining: 2h 22m 33s
9:	test: 0.8093999	best: 0.8093999 (9)	total: 1m 25s	remaining: 2h 21m 40s
10:	test: 0.8109476	best: 0.8109476 (10)	total: 1m 34s	remaining: 2h 21m 36s
11:	test: 0.8120339	best: 0.8120339 (11)	total: 1m 43s	remaining: 2h 21m 27s
12:	test: 0.8200403	best: 0.8200403 (12)	total: 1m 51s	remaining: 2h 21m 1s
13:	test: 0.8527336	best: 0.852733

/tmp/ipython-input-13-2860974315.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top1 = grouped.apply(lambda g: g.loc[g["score"].idxmax()]).reset_index(drop=True)
/tmp/ipython-input-13-2860974315.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topk = grouped.apply(lambda g: g.nlargest(k, "score")).reset_index(drop=True)



Evaluation Metrics (Validation Set):
Accuracy@1 : 0.8612
Recall@3   : 0.9875
MRR        : 0.9246


/tmp/ipython-input-13-2860974315.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mrr = grouped.apply(reciprocal_rank).mean()


Cat boost takes it again, with a larger margin this time. Think this is the way to go. Next thing to do is to finalise the whole pipeline and do one final train.